In [12]:
import sys  
sys.path.insert(0, '../src')
import utils as ut
import rasterio as rs
from rasterio.windows import from_bounds
import numpy as np
import pandas as pd

import os, sys, warnings
from tqdm import tqdm

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [15]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [16]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [17]:
data_path = '../features/'

In [18]:
patches_imgs = ut.list_files_with_absolute_paths(data_path,endswith='stack_ee.tif')
patches_imgs

['../features/p03_stack_ee.tif',
 '../features/p02_stack_ee.tif',
 '../features/p04_stack_ee.tif',
 '../features/p01_stack_ee.tif']

In [19]:
col_names = ['ndvi','ndre','evi','sbi','gvi','wet','satvi','ndmi',"lulc", "blue", "green","red","red_e1","red_e2","red_e3","nir1","swir1","swir2","nir2"]

# Prepare training data

In [20]:
patches_imgs[-3:]

['../features/p02_stack_ee.tif',
 '../features/p04_stack_ee.tif',
 '../features/p01_stack_ee.tif']

In [78]:
df_imgs = [pd.DataFrame(rs.open(f'{img}').read().reshape(19,-1).T,columns=col_names) for img in patches_imgs[-3:]]  
len(df_imgs)

3

In [79]:
df_all = pd.concat(df_imgs)
df_all.shape, df_all.columns

((4506415, 19),
 Index(['ndvi', 'ndre', 'evi', 'sbi', 'gvi', 'wet', 'satvi', 'ndmi', 'lulc',
        'blue', 'green', 'red', 'red_e1', 'red_e2', 'red_e3', 'nir1', 'swir1',
        'swir2', 'nir2'],
       dtype='object'))

In [80]:
df_all.tail()

,ndvi,ndre,evi,sbi,gvi,wet,satvi,ndmi,lulc,blue,green,red,red_e1,red_e2,red_e3,nir1,swir1,swir2,nir2
1059733,0.78243,0.62744,2.18628,2128.44580,892.74548,-430.02701,-1185.73657,0.70341,24.00000,265.00000,330.00000,211.00000,497.00000,1730.00000,2182.00000,2171.00000,871.00000,378.00000,2372.00000
1059734,0.79396,0.62939,2.25899,2200.91162,929.11627,-469.61230,-1210.11499,0.67468,24.00000,256.00000,356.00000,211.00000,507.00000,1739.00000,2210.00000,2229.00000,905.00000,433.00000,2421.00000
1059735,0.80643,0.64031,2.40902,2314.81860,1061.11938,-384.51950,-1190.11182,0.69198,24.00000,259.00000,375.00000,231.00000,530.00000,1788.00000,2186.00000,2417.00000,885.00000,440.00000,2381.00000
1059736,0.80385,0.62212,2.44157,2325.33228,1026.09802,-479.10431,-1263.08972,0.67966,24.00000,260.00000,331.00000,236.00000,557.00000,1937.00000,2322.00000,2391.00000,991.00000,456.00000,2527.00000
1059737,0.00000,0.00000,-0.00000,151405.51562,-29071.32617,-9941.65918,-32767.50000,0.00000,24.00000,65535.00000,65535.00000,65535.00000,65535.00000,65535.00000,65535.00000,65535.00000,65535.00000,65535.00000,65535.00000


In [81]:
df_all.replace([65535,inf,-inf],np.nan,inplace=True)

In [82]:
df_all.isna().sum()

ndvi         0
ndre         0
evi        180
sbi          0
gvi          0
wet          0
satvi        0
ndmi         1
lulc         0
blue      5842
green     5842
red       5842
red_e1    5842
red_e2    5842
red_e3    5842
nir1      5842
swir1     5842
swir2     5842
nir2      5842
dtype: int64

In [84]:
df_all.dropna(inplace=True)
df_all.isna().sum()

ndvi      0
ndre      0
evi       0
sbi       0
gvi       0
wet       0
satvi     0
ndmi      0
lulc      0
blue      0
green     0
red       0
red_e1    0
red_e2    0
red_e3    0
nir1      0
swir1     0
swir2     0
nir2      0
dtype: int64

In [85]:
df_all[['lulc', 'blue', 'green', 'red', 'red_e1', 'red_e2', 'red_e3', 'nir1','swir1', 'swir2', 'nir2']]=\
df_all[['lulc', 'blue', 'green', 'red', 'red_e1', 'red_e2', 'red_e3', 'nir1','swir1', 'swir2', 'nir2']].astype('int64')

In [86]:
df_all.lulc.value_counts(normalize=True)

25   0.25567
12   0.22788
24   0.16733
29   0.09576
21   0.08348
36   0.05973
18   0.04389
23   0.02655
20   0.02311
35   0.00804
3    0.00341
2    0.00303
41   0.00211
Name: lulc, dtype: float64

`23-24-25-29`: `forest (1)`\
`12`: `Non-irrigated arable land (2)`\
`18-20-21`: `agricultural areas (3)`\
`35-36`: `Wetlands (4)`\
`2-3`: `Urban fabric (5)`\
`41`: `Water bodies (6)`

In [87]:
df_all.lulc.replace([23,24,25,29],23,inplace=True) # forest
df_all.lulc.replace([18,20,21],18,inplace=True) # agricultural areas
df_all.lulc.replace([35,36],35,inplace=True) # Wetlands
df_all.lulc.replace([2,3],2,inplace=True) # Urban fabric

In [88]:
df_all.lulc.unique()

array([23, 35, 12, 18,  2, 41])

In [89]:
labels=[23,35,12,18,2,41]
codes=[1,2,3,4,5,6]
labels_dict = {k:i for i,k in zip(codes,labels)}
df_all.lulc = df_all.lulc.map(labels_dict)

In [90]:
df_all.lulc.unique()

array([1, 2, 3, 4, 5, 6])

In [91]:
df_all.to_parquet('../features/training_data_ee_LULC.parquet')

In [92]:
df_all.shape

(4500392, 19)

# Prepare testing data

In [93]:
out_of_sample = pd.DataFrame(rs.open('../features/p03_stack_ee.tif').read().reshape(19,-1).T,columns=col_names)
out_of_sample.shape, out_of_sample.columns

((1874040, 19),
 Index(['ndvi', 'ndre', 'evi', 'sbi', 'gvi', 'wet', 'satvi', 'ndmi', 'lulc',
        'blue', 'green', 'red', 'red_e1', 'red_e2', 'red_e3', 'nir1', 'swir1',
        'swir2', 'nir2'],
       dtype='object'))

In [94]:
out_of_sample.tail()

,ndvi,ndre,evi,sbi,gvi,wet,satvi,ndmi,lulc,blue,green,red,red_e1,red_e2,red_e3,nir1,swir1,swir2,nir2
1874035,0.84997,0.64323,2.29542,3012.89600,1395.21082,-808.27710,-1771.87024,0.66862,23.00000,254.00000,376.00000,203.00000,680.00000,2535.00000,3225.00000,3132.00000,1389.00000,622.00000,3545.00000
1874036,0.86340,0.65630,2.41563,3176.22827,1556.48450,-754.27887,-1810.85461,0.68897,23.00000,248.00000,400.00000,217.00000,702.00000,2650.00000,3371.00000,3383.00000,1399.00000,623.00000,3623.00000
1874037,0.89383,0.69917,2.52629,3693.66162,2074.66162,-636.60321,-1959.29358,0.73016,23.00000,234.00000,438.00000,224.00000,739.00000,2909.00000,3754.00000,4174.00000,1500.00000,651.00000,3920.00000
1874038,0.89505,0.73660,2.36102,3856.20142,2254.59985,-622.29742,-1998.28564,0.72981,23.00000,245.00000,407.00000,196.00000,671.00000,2800.00000,3627.00000,4424.00000,1625.00000,691.00000,3998.00000
1874039,0.88711,0.70675,2.41761,3193.30542,1712.37341,-660.83582,-1756.27905,0.70840,23.00000,211.00000,385.00000,182.00000,606.00000,2655.00000,3215.00000,3527.00000,1391.00000,602.00000,3514.00000


In [95]:
out_of_sample.replace([65535,inf,-inf],np.nan,inplace=True)

In [96]:
out_of_sample.isna().sum()

ndvi         0
ndre         1
evi         52
sbi          0
gvi          0
wet          0
satvi        0
ndmi         1
lulc         0
blue      1496
green     1496
red       1496
red_e1    1496
red_e2    1496
red_e3    1496
nir1      1496
swir1     1496
swir2     1496
nir2      1496
dtype: int64

In [97]:
out_of_sample.dropna(inplace=True)
out_of_sample.isna().sum()

ndvi      0
ndre      0
evi       0
sbi       0
gvi       0
wet       0
satvi     0
ndmi      0
lulc      0
blue      0
green     0
red       0
red_e1    0
red_e2    0
red_e3    0
nir1      0
swir1     0
swir2     0
nir2      0
dtype: int64

In [98]:
out_of_sample[['lulc', 'blue', 'green', 'red', 'red_e1', 'red_e2', 'red_e3', 'nir1','swir1', 'swir2', 'nir2']]=\
out_of_sample[['lulc', 'blue', 'green', 'red', 'red_e1', 'red_e2', 'red_e3', 'nir1','swir1', 'swir2', 'nir2']].astype('int64')

In [99]:
out_of_sample.lulc.replace([23,24,25,29],23,inplace=True) # forest
out_of_sample.lulc.replace([18,20,21],18,inplace=True) # agricultural areas
out_of_sample.lulc.replace([35,36],35,inplace=True) # Wetlands
out_of_sample.lulc.replace([2,3,7],2,inplace=True) # Urban fabric

In [100]:
out_of_sample.lulc.unique()

array([23, 12, 18,  2, 35])

In [101]:
labels=[23,35,12,18,2,41]
codes=[1,2,3,4,5,6]
labels_dict = {k:i for i,k in zip(codes,labels)}
out_of_sample.lulc = out_of_sample.lulc.map(labels_dict)

In [102]:
out_of_sample.lulc.unique()

array([1, 3, 4, 5, 2])

In [103]:
out_of_sample.to_parquet('../features/testing_data_ee_LULC.parquet')

In [104]:
out_of_sample.shape

(1872491, 19)